In [ ]:
import os
\
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"


In [ ]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

In [ ]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2.yaml"
conversation = None
if default_yaml is None:
     conversation = [dict(role="user", content=[dict(type="image"), 
                    dict(type="text", 
                         text="Extract the text from both columns in the image, preserving the structure and formatting, ensure no grammatical correction is performed.")])]

In [ ]:
print(image_path)

In [ ]:
images = sorted(utils.load_images(image_dir))[:10]

In [ ]:
images

In [ ]:
%%time
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 100000, batch_size=1, temperature=0.4)

In [ ]:
model.conversation

In [ ]:
%%time
outputs = model(images, save=False, debug=True)

In [ ]:
image_name, out = outputs[3]

In [ ]:
print(image_name)
print("="*10)
print(out)

In [ ]:
final_message = "\n\n".join([message for _, message in outputs])
final_message += "\n\nMonocotyledone\n\n Sup"
final_message += "\n\nAlgae\n\n Hello World"
final_message += "\n\nMonocotyledone\n\n Sup"

In [ ]:
print(final_message)

In [ ]:
import re
def clean_message(message):

    result = re.sub(r"\*\*(.+?)\*\*", r"\1", message, flags=re.MULTILINE) # Remove any markdown (bold) on strings
    result = re.sub(r"```", "", result, flags=re.MULTILINE) # Remove any markdown
    result = re.sub(r"^Catalogue|catalogue$", "", result, flags=re.MULTILINE) # Remove Catalogue/catalogue
    result = re.sub(r"^[0-9]+$", "", result, flags=re.MULTILINE) # Remove page numbers
    result = re.sub(r"^John Lightfoot$", "", result, flags=re.MULTILINE) # Remove John Lightfoot name if on its own
    result = re.sub(r"\n{3,}", "", result)
    
    return result

def split_division(message, divisions=["Dicotyledones", "Monocotyledone", "Pteridophyta", "Hepaticae", "Algae"]):

    if divisions is None:
        return [("MAIN", message)]
    
    division_str = "|".join(divisions)
    regex = re.compile(f"({division_str})")
    result = re.split(regex, message)

    result = list(filter(None,result))

    return list(zip(result[::2], result[1::2]))

def split_family(message):

    regex = re.compile("\n\n(?=[A-Z ]+\n)")

    result = re.split(regex, message)

    return list(filter(None,result))

def convertToTextBlocks(message, divisions=["Dicotyledones", "Monocotyledone", "Pteridophyta", "Hepaticae", "Algae"]):

    cleaned_message = clean_message(message)

    division_splits = split_division(cleaned_message, divisions)

    splits = {}

    for division, division_text in division_splits:

        if division in splits:
            splits[division].extend(split_family(division_text))
        else:
            splits[division] = split_family(division_text)
    
    return splits


In [ ]:
result = convertToTextBlocks(final_message)

In [ ]:
result

In [ ]:
split_result

In [ ]:
print(family_split[0][9])

In [ ]:
print(result)

In [ ]:
import math

In [ ]:
math.ceil(0.4578)